In [1]:
submission_file_path = 'submission.csv'

In [ ]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

y = train_df['class4'] == 'nonevent'
X = train_df.drop(columns=['class4', 'id', 'date'])
print(train_df['class4'].value_counts())

pipeline = make_pipeline(
    StandardScaler(),
    PCA(n_components=5),
    LogisticRegression(),
)

pipeline.fit(X, y)
X_test = test_df.drop(columns=['id', 'date'])


proba = pipeline.predict_proba(X_test)[:, 1]
prediction = ["nonevent" if p >= 0.5 else "II" for p in proba]

result_df = pd.DataFrame({
    "id": test_df["id"],
    "class4": prediction,
    "p": proba
})

result_df.to_csv(submission_file_path, index=False)


class4
nonevent    225
II          117
Ib           82
Ia           26
Name: count, dtype: int64
